In [2]:
import pickle
import pandas as pd
# from keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
# import re
# import scipy
# import numpy as np
from sklearn import preprocessing
import numpy as np
import re
import scipy

In [3]:
def load_word2vec_model(path):
    if "Model" not in path:
        print("You may not loading a model; You may loading a tokenizer.")
    if "glove".lower() in path.lower().split("/")[-1]:
        with open(path, 'r', encoding='UTF-8') as f:

            words = set()
            # list(tokenizer.word_index.keys())
            word_to_vec_map = {}
            for line in f:
                w_line = line.split()
                curr_word = w_line[0]
                if curr_word in list(tokenizer.word_index.keys()):
                    word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    elif "word2vec".lower() in path.lower().split("/")[-1]:
        word_to_vec_map = pickle.load(open(path, 'rb'))
    elif "google".lower() in path.lower().split("/")[-1]:

        word_to_vec_map = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(path, binary=True)
    elif "doc2vec".lower() in path.lower().split("/")[-1]:
        word_to_vec_map = pickle.load(open(path, 'rb'))
    return word_to_vec_map

def word_vector_avg(inv_tokenizer, fun_word_to_vec_map, tw_sequence, size):
    vec = np.zeros(size)
    count = 0
    columnNameList = []
    for i in range(18):
        columnName = "rule" + str(i + 1)
        columnNameList.append(columnName)
    for seq in tw_sequence:
        try:
            if inv_tokenizer[seq] in columnNameList:
                embedding_vector = np.ndarray(shape=(size,))
                embedding_vector[:] = 1/size
            else:
                embedding_vector = fun_word_to_vec_map[inv_tokenizer[seq]]
            if embedding_vector is not None:
                vec += embedding_vector
            else:
                vec += np.zeros(size)
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [5]:
fileNameWord2vecModel = "22-05-06_20-59-32_Unigrams_CountVect_WRul_Freq-3803"
pathToWord2vecModel = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/word2vec_vect/" + fileNameWord2vecModel

fun_word_to_vec_map = load_word2vec_model(pathToWord2vecModel)
fun_word_to_vec_map

You may not loading a model; You may loading a tokenizer.


UnboundLocalError: local variable 'word_to_vec_map' referenced before assignment

In [36]:
def apply_gen_rules_features(X, X_train_WoR_dtm, tokenizer, X_train_WoR_Features):
    X_Rules_dtm, features_Rules = gen_rules_features(X, tokenizer)
    X_train_WR_dtm, combined_features = concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm,
                                                                      X_train_WoR_Features, features_Rules)
    return X_train_WR_dtm, combined_features

def gen_rules_features(X_data_series, tokenizer):  # , X_data_dtm, features_arg):
    '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
it as features...
I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
  for changing datatypes'''
    X_data_DF = series_to_DataFrame(X_data_series)
    regexes = [
        re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b',
                   re.I | re.M),
        re.compile(r'\b(I\'m|Im)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
        re.compile(r'\b(we\'re|we are)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
        re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b',
                   re.I | re.M),
        re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
        re.compile(r'\b(we\'ll|we will)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
        re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
        re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
        re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
        re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
        re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I | re.M),
        re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I | re.M),
        re.compile(r'\b\w*\s*\b\?', re.I | re.M),
        re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I | re.M),
        re.compile(r'\b(can|could|should).*(you|u)\b', re.I | re.M),
        re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
        re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
        re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M)

    ]
    temp = pd.DataFrame()
    features_arg = []
    for i, regex in zip(range(len(regexes)), regexes):
        # we can also use ruleString()
        columnName = "rule" + str(i + 1)
        features_arg.append(columnName)
        temp[columnName] = X_data_DF['tweet_text'].apply(lambda text: apply_regex(text, regex, tokenizer.word_index[columnName]))

    temp_sparse = scipy.sparse.csr_matrix(temp.values)
    return temp_sparse, features_arg

def series_to_DataFrame(X_data):
    X_data = X_data.to_frame()
    return X_data

def apply_regex(text, regex, ruleTokenizerSequenceIndex):
    match_found = (re.search(regex, text) != None)
    if match_found:
        match_found = int(ruleTokenizerSequenceIndex)
    else:
        match_found = 0
    return match_found


def concat_sparse_matrices_h(data_X_dtm, data_Rules_dtm, features_X, features_Rules):
    combined_features = features_X + features_Rules
    concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
    return concat_sparse, combined_features

In [54]:
def invTokenizer(tokenizer):
    inv_tokenizer = {v: k for k, v in tokenizer.word_index.items()}
    return inv_tokenizer

def word2vec_Vect(pathToTokenizer, pathToWord2vecModel, df, path_to_nonPreprocessedDataset = "", include_rules=False):
    tokenizer = pickle.load(open(pathToTokenizer, 'rb'))
    embed_dim = int(pathToTokenizer.split("Freq-")[1])
    if "WRul" in pathToTokenizer:
        MAX_SEQUENCE_LENGTH = int(pathToTokenizer.split("Seq-")[1].split("_")[0]) - 18
    else:
        MAX_SEQUENCE_LENGTH = int(pathToTokenizer.split("Seq-")[1].split("_")[0])
        
    sequences = tokenizer.texts_to_sequences(df['tweet_text'])
    X_train_WoR_dtm = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    X_train_Features = ["_Not_Apply_"]
    print(type(X_train_WoR_dtm), X_train_WoR_dtm.shape)
    if include_rules == 'yes':
        df_notPreprocessed = pd.read_csv(path_to_nonPreprocessedDataset)
        X_train_dtm, _ = apply_gen_rules_features(df_notPreprocessed["tweet_text"], X_train_WoR_dtm, tokenizer, \
                                                  X_train_Features)
    else:
        X_train_dtm = X_train_WoR_dtm
        X_train_dtm = np.unique(X_train_dtm, axis=0)
    print(type(X_train_dtm), X_train_dtm.shape)
    inv_tokenizer = invTokenizer(tokenizer)
    fun_word_to_vec_map = load_word2vec_model(pathToWord2vecModel)
    
   
    vector = []
    for tw_sequence in X_train_dtm.toarray():
        vec = word_vector_avg(inv_tokenizer, fun_word_to_vec_map, tw_sequence, embed_dim)
        vector.append(vec)
    X_train_dtm = np.array(vector)
    print(50*"*")
    print("Evaluation")
    if "WRul" in pathToTokenizer:
        if include_rules.lower() == "no":
            print("Vectoirzer and appending does not match")
    if "WoRul" in pathToTokenizer:
        if include_rules.lower() == "yes":
            print("Vectoirzer and appending does not match")
            
    if "WRul" in pathToWord2vecModel:
        if include_rules.lower() == "no":
            print("Word2vec Model and appending does not match")
    if "WoRul" in pathToWord2vecModel:
        if include_rules.lower() == "yes":
            print("Word2vec Model and appending does not match")
    print(50*"*")
    return X_train_dtm
  
    

    
filenameTokenizer = "22-04-15_09-10-29_word2vec_Vect_WRul_Max-Len-Seq-38_Freq-300"
pathToTokenizer = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/word2vec_vect/" + filenameTokenizer

fileNameWord2vecModel = "22-04-15_09-10-29_word2vec_Model_WRul_Max-Len-Seq-38_Freq-300"
pathToWord2vecModel = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/word2vec_vect/" + fileNameWord2vecModel

fnameTrainingDataset = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/datasets/preprocessed_500_random_sample.csv"
path_to_nonPreprocessedDataset = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/datasets/500_random_sample.csv"
include_rules = "yes"
df = pd.read_csv(fnameTrainingDataset)
X_train_dtm = word2vec_Vect(pathToTokenizer, pathToWord2vecModel, df,\
                                                    path_to_nonPreprocessedDataset, include_rules)    
X_train_dtm.shape

FileNotFoundError: [Errno 2] No such file or directory: 'F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/datasets/preprocessed_500_random_sample.csv'

In [131]:
def loadClassificationModel(pathToClfModel):
    clf_nb = pickle.load(open(pathToClfModel, 'rb'))
    return clf_nb

fileNameClfModel = "22-04-14_11-27-52_Vect_WRul_Max-Len-Seq-20_Freq-318_22-04-14_11-27-52_word2vec_Vect_WRul_Max-Len-Seq-20_Freq-318_NB"
pathToClfModel = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/word2vec_clfModels/" + fileNameClfModel
clf_nb = loadClassificationModel(pathToClfModel)
clf_nb.predict(X_train_dtm)

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,

In [11]:
fnameTrainingDataset = "datasets/200_random_sample.csv"
df = pd.read_csv(fnameTrainingDataset)
X = df['tweet-text']
y_binary = df['tweet-class']
le = preprocessing.LabelEncoder()
y_binary = le.fit_transform(y_binary)

classes_labels_binary = le.classes_
sequences = load_vect.texts_to_sequences(X)
X_train_WoR_dtm = pad_sequences(sequences, maxlen=38)
X_train_WoR_Features = list(load_vect.word_index.keys())

In [77]:
import numpy as np
# create numpy arrays
data = np.array([[1, 14, 3, 14, 14],
                 [1, 14, 3, 14, 14],
                 [3, 13, 1, 13, 13],
                 [7, 12, 9, 12, 12],
                 [9, 11, 9, 11, 11],
                 [7, 12, 9, 12, 12],
                 [7, 10, 9, 10, 10]])
# Remove Duplicate columns from 2D NumPy Array
print(data)
print(data.shape)
data_uniq, indice_uniq = np.unique(data, return_index=True, axis=0)
print(data_uniq)
print(data_uniq.shape)
print(indice_uniq)
print(np.take(data, indice_uniq))

[[ 1 14  3 14 14]
 [ 1 14  3 14 14]
 [ 3 13  1 13 13]
 [ 7 12  9 12 12]
 [ 9 11  9 11 11]
 [ 7 12  9 12 12]
 [ 7 10  9 10 10]]
(7, 5)
[[ 1 14  3 14 14]
 [ 3 13  1 13 13]
 [ 7 10  9 10 10]
 [ 7 12  9 12 12]
 [ 9 11  9 11 11]]
(5, 5)
[0 2 6 3 4]
[ 1  3 14 14 14]


In [78]:
data

array([[ 1, 14,  3, 14, 14],
       [ 1, 14,  3, 14, 14],
       [ 3, 13,  1, 13, 13],
       [ 7, 12,  9, 12, 12],
       [ 9, 11,  9, 11, 11],
       [ 7, 12,  9, 12, 12],
       [ 7, 10,  9, 10, 10]])

In [86]:
import gensim
path = 'word2vecModels/crisisNLP_word_vector.bin'
# model = word2vec.Word2Vec.load_word2vec_format('../word2vecModels/crisisNLP_word_vector.bin', binary=True)
model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(path, binary=True)
words=model.most_similar(positive=['shelter'], negative=[], topn=10)

FileNotFoundError: [Errno 2] No such file or directory: 'word2vecModels/crisisNLP_word_vector.bin'

In [15]:
filename = "22-04-14_09-37-50_word2vec_Vect_WRul_Max-Len-Seq-38_Freq-318"
path = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/word2vec_vect/" + filename
load_model = pickle.load(open(path, 'rb'))

In [16]:
def apply_regex(text, regex):
    match_found = (re.search(regex ,text) !=None)
    match_found = int(match_found == True)
    return match_found

def series_to_DataFrame(X_data):
    X_data = X_data.to_frame()
    return X_data

def concat_sparse_matrices_h(data_X_dtm, data_Rules_dtm, features_X, features_Rules):
    combined_features = features_X + features_Rules
    concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
    return concat_sparse, combined_features

def gen_rules_features(X_data_series): # , X_data_dtm, features_arg):
    '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
    it as features...
    I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
      for changing datatypes'''
    X_data_DF = series_to_DataFrame(X_data_series)
#     X_data_DF = X_data_series
    regexes = [
      re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I|re.M),
      re.compile(r'\b(I\'m)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I|re.M),
      re.compile(r'\b(we\'re)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I|re.M),
      re.compile(r'\b(we\'ll)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I|re.M),
      re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I|re.M),
      re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I|re.M),
      re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I|re.M),
      re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I|re.M),
      re.compile(r'\b\w*\s*\b\?', re.I|re.M),
      re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I|re.M),
      re.compile(r'\b(can|could|should).*(you|u)\b', re.I|re.M),
      re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I|re.M),
      re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I|re.M),
      re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I|re.M)

    ]
    temp = pd.DataFrame()
    features_arg = []
    for i, regex in zip(range(len(regexes)), regexes):
      columnName = "RegEx_" + str(i + 1)
      features_arg.append(columnName)
      temp[columnName] = X_data_DF['tweet-text'].apply(lambda text: apply_regex(text, regex))
    temp_sparse = scipy.sparse.csr_matrix(temp.values)
    return temp_sparse, features_arg

In [14]:
fnameTrainingDataset = "datasets/200_random_sample.csv"
df = pd.read_csv(fnameTrainingDataset)
X = df['tweet-text']
y_binary = df['tweet-class']
le = preprocessing.LabelEncoder()
y_binary = le.fit_transform(y_binary)

classes_labels_binary = le.classes_
X_train_WoR_dtm = load_vect.transform(X)
X_train_WoR_Features = list(load_vect.get_feature_names_out())

AttributeError: 'Tokenizer' object has no attribute 'transform'

In [173]:
X_Rules_dtm, features_Rules = gen_rules_features(X)
X_train_WR_dtm, combined_features = concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm, X_train_WoR_Features, features_Rules)

In [174]:

y_bin_pred = load_model.predict(X_train_WR_dtm)

In [175]:
import numpy as np

df['DT'] = y_bin_pred.tolist()
df['GB'] = np.random.randint(2, size=500).tolist()
df['MLP'] = np.random.randint(2, size=500).tolist()
df['RF'] = np.random.randint(2, size=500).tolist()

In [134]:
df["request"] = df.iloc[:, -4:].mean(axis=1) > 0.6

In [176]:
df.iloc[:, -4:]

,DT,GB,MLP,RF
0,0,0,1,0
1,1,0,1,1
2,0,1,1,1
3,0,1,1,1
4,0,1,0,0
...,...,...,...,...
495,1,0,0,1
496,0,0,1,1
497,0,0,1,1
498,0,1,0,1


In [358]:
e = np.empty([0])
e = np.append(e, 1)
e = np.append(e, 2)
e = np.append(e, 3)
print(list(e))

[1.0, 2.0, 3.0]


In [345]:
df.iloc[:, -2:].values

array([[1, 0],
       [2, 2],
       [3, 0],
       [0, 0]], dtype=int64)

In [171]:
df.iloc[:, -:]

,GB,MLP,RF,num_uniq
0,0,1,1,4
1,0,0,0,4
2,0,0,0,4
3,1,1,0,4
4,1,1,1,4
...,...,...,...,...
495,0,0,0,4
496,1,0,1,4
497,0,0,0,4
498,0,0,0,4


In [366]:
def get_max_from_rows_into_column(df, indexes):
    """ The function takes dataframe and index of columns, and 
    then it returns the array containg the most frequent value against each row.
    Example: 
    dataframe:
    'c1' | 'c2'| 'c3' | 'c4'
    -------------------------
      1  |  1  |   1  |  0  
      2  |  1  |   2  |  2
      2  |  3  |   3  |  0 
      2  |  0  |   0  |  0 
      returned array:
      (1, 2, 3, 0)
      df: dataframe
      indexes: indexes of columns
      example: get_max_from_rows_into_column(dataframe, -3:)
      """
    def get_array(df):
        max_unique_label_column = np.empty([0])
        for v in df.values:
            unique_label, freq_label = np.unique(v, return_counts=True)
            print(unique_label[np.argmax(freq_label)])
            max_unique_label_column = np.append(max_unique_label_column, unique_label[np.argmax(freq_label)])
            
#             print(unique_label[np.argmax(freq_label)])
        return max_unique_label_column
    if ":" in indexes:
        if indexes[0] == ":":
            
            df = df.iloc[:, :int(indexes[1:])]
            max_unique_label_column = get_array(df)
        if indexes[-1] == ":":
            df = df.iloc[:, int(indexes[:-1]):]
            max_unique_label_column = get_array(df)
           
        else:
            indexes = [int(x) for x in indexes.split(":")]
            df = df.iloc[:, indexes[0]: indexes[1]]
            max_unique_label_column = get_array(df)
    else:
            indexes = [int(x) for x in indexes.split(",")]
            df = df.iloc[:, indexes]
            max_unique_label_column = get_array(df)
    print(max_unique_label_column)
    return max_unique_label_column
#     for v in df.iloc[:, indexes].values:
#         unique_label, freq_label = np.unique(v, return_counts=True)
#         print(unique_label)
#         print(freq_label)
#         print(unique_label[np.argmax(freq_label)])
#     #     print(counts.indexOf(max(counts)))
    #     print(unique)

#         print(40*"*")

d = {'c1': [1, 2, 2, 2], 'c2': [1, 1, 3, 0], 'c3': [1, 2, 3, 0], 'c4': [0, 2, 0, 0]}
df = pd.DataFrame(data=d)

indexes = "-3:"
df["new"] = get_max_from_rows_into_column(df, indexes)
df

1
2
3
0
[1. 2. 3. 0.]


,c1,c2,c3,c4,new
0,1,1,1,0,1.0
1,2,1,2,2,2.0
2,2,3,3,0,3.0
3,2,0,0,0,0.0


In [300]:
indexes = "-4:-3"
print(indexes)
if ":" in indexes:
    if indexes[0] == ":" or indexes[-1] == ":":
        indexes = indexes
    else:
        indexes = [int(x) for x in indexes.split(":")]
        return
else:
    indexes = [int(x) for x in indexes.split(",")]
print(indexes)
df.iloc[:, indexes]


-4:-3
[-4, -3]


,DT,GB
0,0,0
1,1,0
2,0,1
3,0,1
4,0,1
...,...,...
495,1,0
496,0,0
497,0,0
498,0,1


In [335]:
d = {'c1': [1, 2, 2, 2], 'c2': [1, 1, 3, 0], 'c3': [1, 2, 2, 0], 'c4': [0, 2, 0, 0]}
df = pd.DataFrame(data=d)
df

,c1,c2,c3,c4
0,1,1,1,0
1,2,1,2,0
2,2,3,2,0
3,2,0,0,0


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
fnameTrainingDataset = "datasets/500_random_sample.csv"
df = pd.read_csv(fnameTrainingDataset)
X = df['tweet_text']
Vect = CountVectorizer(ngram_range=(2,2), min_df=1, max_df=1)
Vect.fit(X)
X_train_WoR_dtm = Vect.transform(X)
X_train_WoR_dtm

<500x2600 sparse matrix of type '<class 'numpy.int64'>'
	with 2600 stored elements in Compressed Sparse Row format>